# Create DataFrame from different sources

## Create DF from CSV

In [0]:
users_df = spark.read.csv(path="dbfs:/FileStore/synechron/users/csv_format/*.csv",
                          header=True)
users_df.limit(4).display()

id,age,gen,designation,salary
1,26,M,Technician,85711
2,53,F,Other,94043
3,23,M,Writer,32067
4,26,M,Technician,43537


## Create DF from Delimited

In [0]:
users_df = spark.read.csv(path="dbfs:/FileStore/synechron/users/delimited_format/users_001.dat",
                          sep="|",
                          header=True)
users_df.limit(4).display()

id,age,gen,designation,salary
1,26,M,Technician,85711
2,53,F,Other,94043
3,23,M,Writer,32067
4,26,M,technician,43537


## Create DF from JSON

In [0]:
spark.read.json(
    path="dbfs:/FileStore/synechron/users/json_format/users_001.json"
).limit(4).display()

age,designation,gender,id,salary
0,technician,M,1,85711
53,other,F,2,94043
23,writer,M,3,32067
0,technician,M,4,43537


## Create DF from Parquet

In [0]:
spark.read.parquet(
    "dbfs:/FileStore/synechron/users/parquet_format/users.parquet"
).limit(4).display()

id,age,gender,designation,salary
1,0,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,0,M,technician,43537


# Exploring DataFrameReader

## Important Parameters

### Common Options for .csv

In [0]:
users_df = spark.read.csv(path="dbfs:/FileStore/synechron/users/delimited_format/users_001.dat",
                          sep="|",
                          header=True,
                          inferSchema=True)
users_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gen: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- salary: integer (nullable = true)



### Common Options for .json

In [0]:
spark.read.json(
    path="dbfs:/FileStore/synechron/users/json_format/users_003.json",
    multiLine=True
).limit(4).display()

age,designation,gender,id,salary
0,technician,M,1,85711
53,other,F,2,94043


## Attach Custom Schema

In [0]:
from pyspark.sql.types import *

USER_SCHEMA = StructType([
  StructField("user_id", IntegerType()),
  StructField("age", IntegerType()),
  StructField("gen", StringType()),
  StructField("designation", StringType()),
  StructField("salary", IntegerType())
])

users_df = spark.read.csv(path="dbfs:/FileStore/synechron/users/delimited_format/users_001.dat",
                          sep="|",
                          header=True,
                          schema=USER_SCHEMA)
users_df.printSchema()                          

root
 |-- user_id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gen: string (nullable = true)
 |-- designation: string (nullable = true)
 |-- salary: integer (nullable = true)



## Handling Bad Record

### PERMISSIVE (Default)

In [0]:
spark.read.option("columnNameOfCorruptRecord", "bad_records").json(
    path="/FileStore/synechron/access_logs.json", mode="PERMISSIVE"
).display()

bad_records,email,first_name,gender,id,ip_address,last_name
null,jpenddreth0@census.gov,Jeanette,Female,1,26.58.193.2,Penddreth
null,gfrediani1@senate.gov,Giavani,Male,2,229.179.4.212,Frediani
null,nbea2@imageshack.us,Noell,Female,3,180.66.162.255,Bea
"{""id"": 4,""first_name"": {,""last_name"": ""Valek"",""email"": ""wvalek3@vk.com"",""gender"": ""Male"",""ip_address"": ""67.76.188.26""}",null,null,null,null,null,null


### DROPMALFORMED

In [0]:
spark.read.json(
    path="/FileStore/synechron/access_logs.json", mode="DROPMALFORMED"
).display()

email,first_name,gender,id,ip_address,last_name
jpenddreth0@census.gov,Jeanette,Female,1,26.58.193.2,Penddreth
gfrediani1@senate.gov,Giavani,Male,2,229.179.4.212,Frediani
nbea2@imageshack.us,Noell,Female,3,180.66.162.255,Bea


### FAILFAST

In [0]:
spark.read.json(path="/FileStore/synechron/access_logs.json", mode="FAILFAST").display()

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-1026188932982138>:1
----> 1 spark.read.json(path="/FileStore/synechron/access_logs.json", mode="FAILFAST").display()

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:420, in DataFrameReader.json(self, path, schema, primitivesAsString, prefersDecimal, allowComments, allowUnquotedFieldNames, allowSingleQuotes, allowNumericLeadingZero, allowBackslashEscapingAnyCharacter, mode, columnNameOfCorruptRecord, dateFormat, timestampFormat, multiLine, allowUnqu